<a href="https://colab.research.google.com/github/SHEHAN-120/Genai-Video-Summarizer/blob/main/Gen_AI_Video_Summaerizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yt-dlp faster-whisper transformers torch gradio pydub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.8 MB/s eta 0:00:00


In [2]:
import yt_dlp
from faster_whisper import WhisperModel
from transformers import pipeline
import gradio as gr

In [3]:
def download_youtube_audio(youtube_url):
    output_file = "video_audio.mp3"


    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': 'video_audio.%(ext)s',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
    }


    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

    return output_file

In [4]:
model = WhisperModel("tiny", device="cpu", compute_type="int8", cpu_threads=8)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocabulary.txt: 0.00B [00:00, ?B/s]

model.bin:   0%|          | 0.00/75.5M [00:00<?, ?B/s]

In [5]:
def transcribe_audio(audio_path):

    segments, _ = model.transcribe(audio_path)
    transcript = " ".join(segment.text for segment in segments)

    return transcript

In [6]:
def chunk_text(text, chunk_size=400):
    """Splits text into smaller chunks to fit the model's token limit."""
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

In [7]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [8]:
def summarize_text(text):


    text_chunks = chunk_text(text, chunk_size=400)

    summaries = []
    for chunk in text_chunks:
        summary = summarizer(chunk, do_sample=False)
        summaries.append(summary[0]['summary_text'])


    full_summary = " ".join(summaries)

    return full_summary

In [9]:
def process_video(youtube_url):

    audio_path = download_youtube_audio(youtube_url)


    transcript = transcribe_audio(audio_path)


    summary = summarize_text(transcript)

    return summary

In [10]:

with gr.Blocks() as demo:
    gr.Markdown("## 🎥 Generative AI Video Summarizer")
    gr.Markdown("### Paste a YouTube link to generate a summarized transcript.")

    youtube_url = gr.Textbox(label="YouTube Video URL")
    output_text = gr.Textbox(label="Summarized Key Points", lines=10)

    summarize_button = gr.Button("Summarize Video")
    summarize_button.click(process_video, inputs=[youtube_url], outputs=[output_text])


In [11]:
demo.launch(share = True, debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d2e4f95e01d169301b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[youtube] Extracting URL: https://youtu.be/zHhR3daI3bY?si=-9boycyTwXrstmAY
[youtube] zHhR3daI3bY: Downloading webpage
[youtube] zHhR3daI3bY: Downloading android sdkless player API JSON
[youtube] zHhR3daI3bY: Downloading tv client config
[youtube] zHhR3daI3bY: Downloading player 65578ad1-main
[youtube] zHhR3daI3bY: Downloading tv player API JSON
[youtube] zHhR3daI3bY: Downloading web safari player API JSON


         player = https://www.youtube.com/s/player/65578ad1/player_ias.vflset/en_US/base.js
         n = FBndWmt0epxaf0K ; player = https://www.youtube.com/s/player/65578ad1/player_ias.vflset/en_US/base.js
         Please report this issue on  https://github.com/yt-dlp/yt-dlp/issues?q= , filling out the appropriate issue template. Confirm you are on the latest version using  yt-dlp -U


[youtube] zHhR3daI3bY: Downloading m3u8 information
[info] zHhR3daI3bY: Downloading 1 format(s): 251
[download] Sleeping 5.00 seconds as required by the site...
[download] Destination: video_audio.webm
[download] 100% of    1.65MiB in 00:00:00 at 7.61MiB/s   
[ExtractAudio] Destination: video_audio.mp3
Deleting original file video_audio.webm (pass -k to keep)
[youtube] Extracting URL: https://youtu.be/gb-ljOXzpYs?si=AQ4BR7w9-fTotXlS
[youtube] gb-ljOXzpYs: Downloading webpage
[youtube] gb-ljOXzpYs: Downloading android sdkless player API JSON
[youtube] gb-ljOXzpYs: Downloading tv client config
[youtube] gb-ljOXzpYs: Downloading tv player API JSON
[youtube] gb-ljOXzpYs: Downloading web safari player API JSON


[youtube] gb-ljOXzpYs: Downloading m3u8 information
[info] gb-ljOXzpYs: Downloading 1 format(s): 251
[download] Sleeping 5.00 seconds as required by the site...
[download] Destination: video_audio.webm
[download] 100% of    1.92MiB in 00:00:00 at 7.14MiB/s   
[ExtractAudio] Destination: video_audio.mp3
Deleting original file video_audio.webm (pass -k to keep)
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d2e4f95e01d169301b.gradio.live
